# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

#### 3 Этап создания модели

В данном ноутбуке мы сделаем следующее:
* Обработаем и отнормируем признаки
* Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
* Построим модель при помощи LinearRegression
* Обучим модель на основе случайного леса RandomForestRegressor
* Обучим модель с L1 и L2 регуляризаций ElasticNetCV
* Сделаем  модель стохастического градиентного спуска SGDRegressor
* На основе предыдущих шагов выберем оптимальную модель

In [33]:
import random
import numpy as np 
import pandas as pd 
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
#отключим оповещения
import warnings
warnings.filterwarnings("ignore")


In [3]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [4]:
df = pd.read_csv('data/data_model.csv')
display(df.head())
df.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,Active,2.0,Mason,3588,50401,IA,244900,False,single_family_home,1970,True,True,False,False,3.8,5.60
3,Other,3.0,Houston,1930,77080,TX,311995,False,single_family_home,2019,True,True,True,False,3.0,0.60
4,For Sale,2.0,Flushing,1300,11354,NY,669000,False,condo,1965,False,False,True,False,2.8,0.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210669 non-null  object 
 1   baths                210669 non-null  float64
 2   city                 210669 non-null  object 
 3   sqft                 210669 non-null  int64  
 4   zipcode              210669 non-null  int64  
 5   state                210669 non-null  object 
 6   target               210669 non-null  int64  
 7   pool_encoded         210669 non-null  bool   
 8   Type                 210669 non-null  object 
 9   Year built           210669 non-null  object 
 10  Heating_encoded      210669 non-null  bool   
 11  Cooling_encoded      210669 non-null  bool   
 12  Parking_encoded      210669 non-null  bool   
 13  fireplace_encoded    210669 non-null  bool   
 14  school_rating _mean  210669 non-null  float64
 15  school_dist_min  

In [5]:
#исправим автоопределение категориальной переменной как числа
df['zipcode'] = df['zipcode'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210669 non-null  object 
 1   baths                210669 non-null  float64
 2   city                 210669 non-null  object 
 3   sqft                 210669 non-null  int64  
 4   zipcode              210669 non-null  object 
 5   state                210669 non-null  object 
 6   target               210669 non-null  int64  
 7   pool_encoded         210669 non-null  bool   
 8   Type                 210669 non-null  object 
 9   Year built           210669 non-null  object 
 10  Heating_encoded      210669 non-null  bool   
 11  Cooling_encoded      210669 non-null  bool   
 12  Parking_encoded      210669 non-null  bool   
 13  fireplace_encoded    210669 non-null  bool   
 14  school_rating _mean  210669 non-null  float64
 15  school_dist_min  

In [6]:
# Составим список булевых признаков:
bin_features = ['pool_encoded','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','zipcode','state','Type','Year built']
 
# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [7]:
# подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = df[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1522
Количество уникальных значений в категориальной колонке zipcode: 3962
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [11]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
 
    # Нормализация данных и логорифмирование
    scaler = MinMaxScaler()
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # логорифмирование ухудшило результат
        #df_output[column] = np.log(df_output[column])
        
    # ################### Categorical Features ############################################################## 
 
    ohe_status = OneHotEncoder(sparse=False)
    ohe_state = OneHotEncoder(sparse=False)
    ohe_Type = OneHotEncoder(sparse=False)

    status_ohe = ohe_status.fit_transform(df_output['status'].values.reshape(-1,1))
    state_ohe = ohe_state.fit_transform(df_output['state'].values.reshape(-1,1))
    Type_ohe = ohe_Type.fit_transform(df_output['Type'].values.reshape(-1,1))

    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])

    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])

    te = TargetEncoder()
    cbe = CatBoostEncoder()

    city_te = te.fit_transform(df_output['city'], df_output['target'])
    city_cbe = cbe.fit_transform(df_output['city'], df_output['target'])

    zip_te = te.fit_transform(df_output['zipcode'], df_output['target'])
    zip_cbe = cbe.fit_transform(df_output['zipcode'], df_output['target'])

    # Adding encoded categorical features to the output dataframe
    df_output = df_output.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_te'] = city_te
    df_output['city_cbe'] = city_cbe
    df_output['zip_te'] = zip_te
    df_output['zip_cbe'] = zip_cbe

    # Dropping original categorical columns
    df_output.drop(['status', 'state', 'Type', 'city', 'zipcode','Year built'], axis=1, inplace=True)
    
    return df_output



In [12]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(df)
df_encoded.sample(10)

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,...,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_te,city_cbe,zip_te,zip_cbe
4168,0.034483,0.009040,0.380718,False,False,False,False,False,0.43,0.044910,...,0.0,0.0,1.0,0.0,32,204,0.434298,0.386283,0.445983,0.384498
33963,0.068966,0.015475,0.536716,False,True,True,True,False,0.80,0.041916,...,0.0,0.0,1.0,0.0,20,159,0.568478,0.588749,0.664729,0.644326
193545,0.068966,0.009396,0.301055,False,True,False,True,False,0.60,0.104790,...,0.0,0.0,0.0,1.0,28,201,0.393095,0.394302,0.394243,0.397593
58797,0.034483,0.009316,0.252438,False,True,True,False,True,0.80,0.056886,...,0.0,0.0,1.0,0.0,28,201,0.291978,0.314284,0.334464,0.339597
108590,0.000000,0.004893,0.445928,False,True,True,True,False,0.48,0.053892,...,0.0,0.0,0.0,0.0,30,189,0.633082,0.612994,0.476207,0.632483
3209,0.068966,0.011785,0.766355,False,True,True,True,False,0.37,0.046707,...,0.0,0.0,1.0,0.0,1,146,0.584631,0.540039,0.648250,0.540039
7142,0.000000,0.007407,0.277704,False,True,False,False,False,0.50,0.041916,...,0.0,0.0,1.0,0.0,28,101,0.420480,0.415779,0.511998,0.497164
161298,0.068966,0.011746,0.236315,False,True,True,True,False,0.60,0.036228,...,0.0,0.0,1.0,0.0,9,163,0.338161,0.337674,0.322207,0.256785
114587,0.068966,0.013057,0.385848,False,True,True,True,False,0.60,0.065868,...,0.0,0.0,1.0,0.0,18,187,0.449768,0.449449,0.333381,0.329613
19770,0.103448,0.015107,0.712951,False,True,True,False,False,0.52,0.056886,...,1.0,0.0,0.0,0.0,28,202,0.368139,0.360930,0.534711,0.548442


In [13]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 74 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    210669 non-null  float64
 1   sqft                     210669 non-null  float64
 2   target                   210669 non-null  float64
 3   pool_encoded             210669 non-null  bool   
 4   Heating_encoded          210669 non-null  bool   
 5   Cooling_encoded          210669 non-null  bool   
 6   Parking_encoded          210669 non-null  bool   
 7   fireplace_encoded        210669 non-null  bool   
 8   school_rating _mean      210669 non-null  float64
 9   school_dist_min          210669 non-null  float64
 10  status_Active            210669 non-null  float64
 11  status_Auction           210669 non-null  float64
 12  status_Back on Market    210669 non-null  float64
 13  status_Coming Soon       210669 non-null  float64
 14  stat

Разделим датасет

In [21]:
y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Обучение модели

## Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю стоимость по общей площади и городу. 
C ней будем сравнивать другие модели.

In [46]:
# Наивная модель
class NaiveModel:
    def __init__(self):
        self.means = None

    def fit(self, X, y):
        X_df = pd.DataFrame(X, columns=['city_te', 'city_cbe'])
        y_df = pd.DataFrame(y, columns=['target'])
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby(['city_te', 'city_cbe'])['target'].mean().reset_index()

    def predict(self, X):
        X = pd.DataFrame(X, columns=['city_te', 'city_cbe']).copy()
        X['mean'] = np.nan
        for idx, row in self.means.iterrows():
            X.loc[(X['city_te'] == row['city_te']) & (X['city_cbe'] == row['city_cbe']), 'mean'] = row['target']
        
        X['mean'].fillna(X['mean'].mean(), inplace=True)
        return X['mean'].to_numpy()

naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")


Train MSE: 0.09
Test MSE: 0.05
Train MAE: 0.24
Test MAE: 0.19
Train R2: -0.79
Test R2: -0.03


Среднеквадратичная ошибка (MSE) - это мера разности между правильными ответами и предсказаниями модели. Чем ниже значение MSE, тем лучше модель предсказывает данные. Ваша модель имеет значение 0,09 на обучающей выборке и 0,05 на тестовой выборке.

Средняя абсолютная ошибка (MAE) также меряет разницу между правильными ответами и предсказаниями, но без возведения в квадрат. Значения 0,24 и 0,19 для обучающей и тестовой выборок соответственно свидетельствуют о средней абсолютной ошибке предсказания модели.

Коэффициент детерминации (R2) - это мера, которая показывает, насколько хорошо предсказания модели объясняют изменчивость примеров. Значения R2 варьируются от -∞ до 1, где 1 означает идеальное совпадение. Отрицательные значения R2 указывают на то, что модель очень плохо предсказывает данные. В вашем случае модель имеет R2 -0,79 на обучающей выборке и -0,03 на тестовой выборке.

Интерпретация результатов: модель показывает недостаточные предсказательные способности на обучающей выборке, особенно в виду отрицательного коэффициента детерминации (R2). Однако на тестовой выборке показатели немного лучше, со значениями MSE и MAE меньше, чем на обучающей выборке, и R2, приближающимся к нулю. Это может указывать на проблему с переобучением модели

## Model 2: LinearRegression

In [15]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.02
Test MSE: 0.02
Train MAE: 0.10
Test MAE: 0.10
Train R2: 0.61
Test R2: 0.61


MSE (Mean Squared Error) - средняя квадратичная ошибка. Меньшие значения MSE указывают на лучшую точность модели. Значение Train MSE = 0.02 свидетельствует о средней ошибке модели на обучающей выборке, а Test MSE = 0.02 - на тестовой выборке. Значения ошибок очень близки, что указывает на адекватное обобщение модели.

MAE (Mean Absolute Error) - средняя абсолютная ошибка. Этот показатель также отражает точность модели и чувствителен к выбросам. Train MAE = 0.10 сообщает о средней ошибке модели на обучающей выборке, а Test MAE = 0.10 - на тестовой выборке. Ошибки также близки для обоих выборок, что хорошо для модели.

R2 (коэффициент детерминации) - показатель, который отражает долю дисперсии зависимой переменной объясненной моделью. Значения R2 колеблются от 0 до 1. Чем ближе значение к 1, тем лучше модель предсказывает зависимую переменную. Train R2 = 0.61 и Test R2 = 0.61 указывают на то, что модель объясняет около 61% дисперсии зависимой переменной на обучающей и тестовой выборке.

В целом, результаты свидетельствуют о том, что модель демонстрирует среднею точность прогнозирования и адекватное обобщение на тестовых данных. Нет признаков переобучения или недообучения. Тем не менее, зависимость от объясняющих переменных составляет 61%, что оставляет место для улучшения предсказательной мощи модели. Это можно сделать, добавив новые переменные или методы оптимизации модели

## Model 2.1: LinearRegression с логарифмированием целевой переменной

In [ ]:
# создаём модель линейной регрессии
#model_log = LinearRegression(fit_intercept=False)

# логарифмирование целевой переменной
# во избежании ошибок исключим нулевые и отрицательные значения из данных перед логарифмическим преобразованием.
#epsilon = 1e-6
#y_train = np.maximum(y_train, epsilon)
#y_test = np.maximum(y_test, epsilon)
#y_train_log = np.log(y_train)
#y_test_log = np.log(y_test)

# вычисляем коэффициенты регрессии
#model_log.fit(X_train, y_train_log)

# делаем предсказания с помощью модели
#y_train_log_pred = model_log.predict(X_train)
#y_test_log_pred = model_log.predict(X_test)

# восстанавливаем исходные значения целевой переменной
#y_train_pred = np.exp(y_train_log_pred)
#y_test_pred = np.exp(y_test_log_pred)

# вычисляем требуемые метрики
#mse_train = metrics.mean_squared_error(y_train, y_train_pred)
#mse_test = metrics.mean_squared_error(y_test, y_test_pred)
#mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
#mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
#r2_train = metrics.r2_score(y_train, y_train_pred)
#r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
#print(f"Train MSE: {mse_train:.2f}")
#print(f"Test MSE: {mse_test:.2f}")
#print(f"Train MAE: {mae_train:.2f}")
#print(f"Test MAE: {mae_test:.2f}")
#print(f"Train R2: {r2_train:.2f}")
#print(f"Test R2: {r2_test:.2f}")


* Train MSE: 0.04
* Test MSE: 0.03
* Train MAE: 0.11
* Test MAE: 0.12
* Train R2: 0.17
* Test R2: 0.33

В целом, результаты показывают, что модель после логарифмирования целевой переменной стала хуже

## Model 3: RandomForestRegressor

In [22]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.00
Test MSE: 0.01
Train MAE: 0.02
Test MAE: 0.06
Train R2: 0.98
Test R2: 0.83


В целом, модель показывает хорошие результаты на обучающей выборке, с низкими значениями MSE и MAE, и высоким R2. Однако, на тестовой выборке метрики немного хуже, что может указывать на переобучение - модель хорошо применяет свои наработки к обучающим данным, но менее успешна для новых данных.

Тем не менее, коэффициент детерминации на тестовой выборке равен 0.83, что говорит о том, что модель все равно оценивает данные относительно хорошо. В зависимости от конкретной задачи и требований к точности, эти результаты могут быть приемлемыми

## Model 4: ElasticNetCV

In [28]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
model_el.fit(X_train, y_train)

# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)

# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.02
Test MSE: 0.02
Train MAE: 0.12
Test MAE: 0.12
Train R2: 0.54
Test R2: 0.54


Так как значения ошибки и коэффициента детерминации одинаковы для обучающей и тестовой выборок, мы можем сказать, что модель не переобучена или недообучена. Однако, коэффициент детерминации равный 0.54 указывает на то, что модель объясняет только около 54% изменчивости зависимой переменной

## Model 5: SGDRegressor

In [34]:
# Создание и обучение модели градиентного спуска без подбора гиперпараметров
#model_sgd = SGDRegressor(random_state=RANDOM_SEED)
#model_sgd.fit(X_train, y_train)

# Предсказание значений на тренировочном и тестовом наборах
#y_train_pred = model_sgd.predict(X_train)
#y_test_pred = model_sgd.predict(X_test)

# Расчет метрик
#mse_train = mean_squared_error(y_train, y_train_pred)
#mse_test = mean_squared_error(y_test, y_test_pred)
#mae_train = mean_absolute_error(y_train, y_train_pred)
#mae_test = mean_absolute_error(y_test, y_test_pred)
#r2_train = r2_score(y_train, y_train_pred)
#r2_test = r2_score(y_test, y_test_pred)

# Вывод результатов
#print(f"Train MSE: {mse_train:.2f}")
#print(f"Test MSE: {mse_test:.2f}")
#print(f"Train MAE: {mae_train:.2f}")
#print(f"Test MAE: {mae_test:.2f}")
#print(f"Train R2: {r2_train:.2f}")
#print(f"Test R2: {r2_test:.2f}")

Train MSE: 1950146658224079382773760.00
Test MSE: 1945784524795032165679104.00
Train MAE: 1367703995238.82
Test MAE: 1365927716077.47
Train R2: -38710304943173374614437888.00
Test R2: -38654499143432082475188224.00


* Train MSE: 1950146658224079382773760.00
* Test MSE: 1945784524795032165679104.00
* Train MAE: 1367703995238.82
* Test MAE: 1365927716077.47
* Train R2: -38710304943173374614437888.00
* Test R2: -38654499143432082475188224.00

Результаты говорят о том, что использованная модель показывает очень слабую производительность и недостаточную предсказательную способность.

Мы даже не будем добовлять ее в сравнение

In [35]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.09, 0.05, 0.24, 0.19, -0.79, -0.03],
        'LinearRegression': [0.02, 0.02, 0.10, 0.10, 0.61, 0.61],
        'RandomForestRegressor': [0.00, 0.01, 0.02, 0.06, 0.98, 0.83],
        'ElasticNetCV': [0.02, 0.02, 0.12, 0.12, 0.54, 0.54]}

df_metric = pd.DataFrame(data)
df_metric.head()


,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV
0,Train MSE,0.09,0.02,0.00,0.02
1,Test MSE,0.05,0.02,0.01,0.02
2,Train MAE,0.24,0.10,0.02,0.12
3,Test MAE,0.19,0.10,0.06,0.12
4,Train R2,-0.79,0.61,0.98,0.54


Исходя из предоставленных данных, можно сделать следующий общий вывод относительно различных моделей, применяемых для решения задачи регрессии:

* NaiveModel показывает самые низкие значения Test R2 и малую точность. Эта модель не подходит для прогнозирования данных в рассматриваемой задаче.

* LinearRegression и ElasticNetCV показывают схожие результаты и умеренную точность с Test MSE равным 0.02 и Test R2 равным 0.61 и 0.54 соответственно. Однако, между ними предпочтительнее использовать модель LinearRegression, так как она имеет более низкие значения ошибок (Train MAE и Test MAE).

* RandomForestRegressor демонстрирует лучшие показатели среди всех моделей с минимальным значением ошибок (Test MSE 0.01) и наиболее высоким коэффициентом детерминации (Test R2 0.83).
В результате, RandomForestRegressor является наиболее подходящей моделью для прогнозирования данных в данной регрессионной задаче, обеспечивая относительно высокую точность и наименьшие ошибки.

Хотелось бы попробовать обучить модель на полиномиальных признаках но из за большого объема данных крашится ядро, так же не хватает вычислительной мощьности для подбора гиперпараметров для модели гардиентного спуска пришлось работать без них

In [37]:
df_encoded.to_csv('data/data_model_ecod.csv', index=False)